# not sure about the implementation

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from dataset import TCNDataset
from model import TCN , MultiStageTCN , ParallelTCNs

In [4]:
def train(model,dataloader,optimizer):
    i=0
    criterion = nn.CrossEntropyLoss()
    running_loss = 0 
    for features, labels, masks in dataloader:
        out = model(features,masks)
        optimizer.zero_grad()
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
                print("    Batch {}: loss = {}".format(i ,loss.item()))
        i += 1
        running_loss = loss.item()
    return running_loss / len(dataloader)


    
def train_parallel(model, dataloader,optimizer):
    model.train()
    i=0
    criterion = nn.CrossEntropyLoss()
    running_loss = 0 
    for features, labels, masks in dataloader:
        out1, out2, out3 ,out_average = model(features,masks)
        optimizer.zero_grad()
        loss1 = criterion(out1, labels)
        loss2 = criterion(out2, labels)
        loss3 = criterion(out3, labels)
        loss_average = criterion(out_average, labels)
        loss = loss1 + loss2 + loss3 + loss_average
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
                print("    Batch {}: combined loss = {}".format(i ,loss.item()))
        i += 1
        running_loss = loss.item()
    return running_loss / len(dataloader)



# function for zero padding for dataloader because of variable video length
# inspired by the code from the paper
def collate_fn_padd(batch):
        batch_input , batch_target = [list(t) for t in zip(*batch)] 
        length_of_sequences = list(map(len, batch_target))
        batch_input_tensor = torch.zeros(len(batch_input), np.shape(batch_input[0])[0], max(length_of_sequences), dtype=torch.float)
        
        batch_target_tensor = torch.ones(len(batch_input), max(length_of_sequences), dtype=torch.long)*(-100)
        
        mask = torch.zeros(len(batch_input), num_classes, max(length_of_sequences), dtype=torch.float)
        
        for i in range(len(batch_input)):
            batch_input_tensor[i, :, :np.shape(batch_input[i])[1]] = batch_input[i]
            
            batch_target_tensor[i, :np.shape(batch_target[i])[0]] = batch_target[i]
            
            mask[i, :, :np.shape(batch_target[i])[0]] = torch.ones(num_classes, batch_target[i].shape[0])
            
        return batch_input_tensor, batch_target_tensor, mask
                        

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 4
epochs = 50
num_classes = 48

In [6]:
training_dataset = TCNDataset(training=True)
training_dataloader = torch.utils.data.DataLoader(training_dataset,collate_fn=collate_fn_padd,  batch_size=batch_size, shuffle=True, drop_last=False)

test_dataset = TCNDataset(training=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset,collate_fn=collate_fn_padd,  batch_size=batch_size, shuffle=False, drop_last=False)

multi_stage_TCN = MultiStageTCN()
multi_stage_TCN_optimizer = torch.optim.Adam(multi_stage_TCN.parameters(),lr=0.001)

Repeat Question 2 with an additional video-level loss.
# 1. The new loss computes the binary cross-entropy between a multi-class video level prediction and a multi-class target that indicates which classes are present in the video.

# The target is a vector with a dimension equals the number of classes in the dataset.  The i-th element of this vector is 1 if the i-th class is present in the video, otherwise it should be 0. 

# To get the video level prediction apply a max pooling on the temporal dimension of the predicted frame-wise logits. 

# Note that the final loss function is the sum of the video level loss and the frame-wise cross-entropy loss that is used in Question 2.

In [7]:
criterion = nn.CrossEntropyLoss()
running_loss = 0 
for features, labels, masks in training_dataloader:
        out = multi_stage_TCN(features,masks)
        multi_stage_TCN_optimizer.zero_grad()
        loss = criterion(out, labels)# +video_level_loss(out, labels)
        print(loss.shape)
        print(loss)
        loss.backward()
        multi_stage_TCN_optimizer.step()
        break

torch.Size([])
tensor(3.8380, grad_fn=<NllLoss2DBackward>)


In [8]:
labels.shape

torch.Size([4, 915])

In [9]:
labels.numpy()

array([[   0,    0,    0, ..., -100, -100, -100],
       [   0,    0,    0, ..., -100, -100, -100],
       [   0,    0,    0, ..., -100, -100, -100],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int64)

# get_target_vector:
The target is a vector with a dimension equals the number of classes in the dataset. The i-th element of this vector is 1 if the i-th class is present in the video, otherwise it should be 0. 

In [78]:


# get target vector
# The target is a vector with a dimension equals the number of classes in the dataset. 
# The i-th element of this vector is 1 if the i-th class is present in the video, otherwise it should be 0. 
def get_target_vector(labels):
    target = torch.zeros((labels.shape[0],48)) # labels 0-47
    i = 0
    
    available_classes = [torch.unique(t) for t in labels]
    available_classes_non_negative = [ torch.nn.functional.relu(ac) for ac in available_classes]
    for i in range(labels.shape[0]):
        target[i][available_classes_non_negative[i]] = 1

    return(target) 

In [79]:
t=get_target_vector(labels)

In [80]:
t.shape

torch.Size([4, 48])

In [81]:
t

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

# Get the video level prediction:
Apply a max pooling on the temporal dimension of the predicted frame-wise logits. 
Question: Size of the max pooling? Is it necessary to apply softmax in the output?

In [14]:
#To get the video level prediction apply a max pooling on
#the temporal dimension of the predicted frame-wise logits.
def get_video_level_prediction(out):
    # Max pool in all the frames?????
    m = nn.MaxPool1d(out.shape[2])
    o = m(out)
    return (o)

In [15]:
m = nn.MaxPool1d(out.shape[2])
o = m(out)

In [16]:
out.shape

torch.Size([4, 48, 915])

In [17]:
o.shape

torch.Size([4, 48, 1])

# Def video_level_loss(out, labels)

In [18]:
def video_level_loss(input,target): #video_prediction, target_vector
    return (torch.nn.functional.binary_cross_entropy(input, target))

In [19]:
input_ = get_video_level_prediction(out)
target_ = get_target_vector(labels)
video_loss = video_level_loss(input,target)

NameError: name 'target' is not defined